In [25]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import math
import open3d as o3d
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.linear_model import RANSACRegressor
from sklearn.cluster import KMeans
from scipy.optimize import minimize
from pathlib import Path


In [6]:
def calculate_normal_vector(points):
    pca = PCA(n_components=3)
    pca.fit(points)
    normal_vector = pca.components_[-1]  # The normal vector is the eigenvector with the smallest eigenvalue

    normal_vector = np.array(normal_vector)
    angle_with_x_axis = np.arccos(np.dot(normal_vector, [1, 0, 0]))
    angle_with_y_axis = np.arccos(np.dot(normal_vector, [0, 1, 0]))
    angle_with_z_axis = np.arccos(np.dot(normal_vector, [0, 0, 1]))
    normal_vector = [angle_with_x_axis, angle_with_y_axis, angle_with_z_axis]
    # print(normal_vector)
    return normal_vector

def angle_with_z_axis(vector):
    vector = vector / np.linalg.norm(vector)
    return np.arccos(np.dot(vector, [0, 0, -1]))

In [5]:
def xy2radius(x, y):
    return math.sqrt(x**2 + y**2)

def xy2theta(x, y):
    return math.atan2(y, x)

In [7]:
def apply_gabor_filter(image, ksize=13, sigma=1, theta=0, lambd=7, gamma=0.5):
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma)
    return cv2.filter2D(image, cv2.CV_32F, kernel)

def cluster_images(images, n_clusters):
    flattened_images = [img.flatten() for img in images]
    kmeans = KMeans(n_clusters=n_clusters).fit(flattened_images)
    return kmeans.labels_, kmeans.cluster_centers_

In [248]:
class Zone:
    def __init__(self):
        self.points = []

class CZM:
    def __init__(self, pcd_file_path):
        self.pcd_file_path = pcd_file_path
        self.point_cloud = None

        # Define range parameters
        self.min_range_ = 4
        self.min_range_z2_ = 12
        self.min_range_z3_ = 28
        self.min_range_z4_ = 60
        self.max_range_ = 100.0
        self.min_ranges_ = [self.min_range_, self.min_range_z2_, self.min_range_z3_, self.min_range_z4_]

        # Define zone, sector, and ring parameters
        self.num_zones = 4
        self.num_sectors_each_zone = [72, 48, 48, 64]
        self.num_rings_each_zone = [4, 4, 5, 5]
        self.ring_sizes_ = [
            (self.min_range_z2_ - self.min_range_) / self.num_rings_each_zone[0],
            (self.min_range_z3_ - self.min_range_z2_) / self.num_rings_each_zone[1],
            (self.min_range_z4_ - self.min_range_z3_) / self.num_rings_each_zone[2],
            (self.max_range_ - self.min_range_z4_) / self.num_rings_each_zone[3]
        ]
        self.sector_sizes_ = [
            2 * math.pi / self.num_sectors_each_zone[0],
            2 * math.pi / self.num_sectors_each_zone[1],
            2 * math.pi / self.num_sectors_each_zone[2],
            2 * math.pi / self.num_sectors_each_zone[3]
        ]
        
        self.images_to_cluster = []
        self.labels = []
        self.max_stripes_list = []

        # Load point cloud
        self.load_point_cloud()
        
    def load_point_cloud(self):
        pcd = o3d.io.read_point_cloud(self.pcd_file_path)
        all_points = np.asarray(pcd.points)
        z_values = [pt[2] for pt in all_points]
        min_z = min(z_values)
        max_z = max(z_values)
        
        # 해당 값을 나중에 다른 메서드에서 사용하기 위해 인스턴스 변수로 저장
        self.point_cloud = all_points
        self.min_z = min_z
        self.max_z = max_z
        print(min_z, max_z)

        # 원본 포인트 클라우드를 시각화
        # o3d.visualization.draw_geometries([pcd])
        
    def calculate_normal_vector(self, points):
        pca = PCA(n_components=3)
        pca.fit(points)
        normal_vector = pca.components_[-1]

        normal_vector = np.array(normal_vector)
        angle_with_x_axis = np.arccos(np.dot(normal_vector, [1, 0, 0]))
        angle_with_y_axis = np.arccos(np.dot(normal_vector, [0, 1, 0]))
        angle_with_z_axis = np.arccos(np.dot(normal_vector, [0, 0, 1]))
        normal_vector = [angle_with_x_axis, angle_with_y_axis, angle_with_z_axis]
        # print(normal_vector)
        return normal_vector

    def angle_with_z_axis(self, vector):
        vector = vector / np.linalg.norm(vector)
        return np.arccos(np.dot(vector, [0, 0, -1]))

    # def apply_gabor_filter(self, image, ksize=7, sigma=1, theta=0, lambd=7, gamma=0.5):
    #     kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma)
    #     # return cv2.filter2D(image, cv2.CV_32F, kernel)
    #     return cv2.filter2D(image, -1, kernel)
        
    def apply_gabor_filter(self,image, angle):
        ksize = 15
        sigma = 5
        lambd = 3
        gamma = 0.5
        psi = 0  # phase offset
        theta = angle * np.pi / 8

        gabor_filter = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi)
        
        filtered_image = cv2.filter2D(image, cv2.CV_8UC3, gabor_filter)

        # 임계값 설정 및 스트라이프 영역 검출
        threshold = np.max(filtered_image) * 0.9
        stripe_regions = filtered_image > threshold
        stripe_image = np.uint8(stripe_regions * 255)

        return stripe_image
        # return filtered_image

    def count_stripes(self,image, min_length=5):
        contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        count = 0
        for contour in contours:
            length = cv2.arcLength(contour, True) # 외곽선의 길이 계산
            if length > min_length: # 길이가 임계값보다 크면 카운트
                count += 1
        return count

    # def extract_features(self, image):
    #     features = []

    #     for angle in range(8):
    #         filtered_image = self.apply_gabor_filter(image, angle)
    #         features.append(np.mean(filtered_image))
    #         features.append(np.std(filtered_image))
    #     return features

    def extract_features(self, image, image_sector_index):
        max_stripes = 0
        best_image = None

        for angle in range(8):
            filtered_image = self.apply_gabor_filter(image, angle)
            num_stripes = self.count_stripes(filtered_image)
            if num_stripes > max_stripes:
                max_stripes = num_stripes
                best_image = filtered_image

        if best_image is not None:
            # print(max_stripes)
            
            self.images_to_cluster.append(best_image.flatten())
            self.labels.append(image_sector_index)
            self.max_stripes_list.append(max_stripes)
        else :
            best_image = np.zeros((15, 15), dtype=np.uint8)
            self.images_to_cluster.append(best_image.flatten())
            self.labels.append(image_sector_index)
            self.max_stripes_list.append(max_stripes)

    def remove_specific_pattern(self, image, threshold):
        # 패딩 추가 (상, 하, 좌, 우 1픽셀씩)
        padded_image = np.pad(image, pad_width=1, mode='constant', constant_values=0)

        # 원래 이미지와 패딩된 이미지의 차이 계산 (상하좌우)
        diff_up = padded_image[:-2, 1:-1] - padded_image[1:-1, 1:-1]
        diff_down = padded_image[2:, 1:-1] - padded_image[1:-1, 1:-1]
        diff_left = padded_image[1:-1, :-2] - padded_image[1:-1, 1:-1]
        diff_right = padded_image[1:-1, 2:] - padded_image[1:-1, 1:-1]

        # 조건에 맞는 픽셀 찾기 (상하좌우 차이가 모두 threshold보다 큰 경우)
        condition = (diff_up > threshold) & (diff_down > threshold) & (diff_left > threshold) & (diff_right > threshold)

        # 조건에 맞는 픽셀을 0으로 설정
        result_image = np.copy(image)
        result_image[condition] = 0

        return result_image

    # def count_stripes(self, image, theta):
    #     filtered_image = self.apply_gabor_filter(image, ksize=13, sigma=1, theta=theta, lambd=7, gamma=0.5)
    #     local_maxima = (filtered_image == cv2.dilate(filtered_image, np.ones((3,3))))
    #     return np.sum(local_maxima)
    # def count_stripes(self, image, theta):
    #     filtered_image = self.apply_gabor_filter(image, ksize=15, sigma=15, theta=theta, lambd=2, gamma=1)
    #     local_maxima = (filtered_image == cv2.dilate(filtered_image, np.ones((3,3))))
    #     total_stripes = np.sum(local_maxima)
    #     if total_stripes == 0:
    #         return 0, None  # 또는 적절한 이미지로 대체 가능합니다.
    #     return total_stripes, filtered_image


    # def count_stripes(self, image, theta):
    #     ksizes = [3, 5, 7, 9, 11, 13]
    #     lambdas = [0.1,0.5,1,3,5,7,9]
    #     thetas = [np.pi * i / 4 for i in range(8)]
    #     total_stripes_all = []

    #     for ksize in ksizes:
    #         for lambd in lambdas:
    #             for theta in thetas:
    #                 filtered_image = self.apply_gabor_filter(image, ksize=ksize, sigma=15, theta=theta, lambd=lambd, gamma=1)
    #                 local_maxima = (filtered_image == cv2.dilate(filtered_image, np.ones((3,3))))
    #                 total_stripes = np.sum(local_maxima)
    #                 total_stripes_all.append(total_stripes)

    #     if all(stripe == 0 for stripe in total_stripes_all):
    #         return 0, None
    #     return total_stripes_all, filtered_image


    # def analyze_cluster(self, cluster_images):
    #     thetas = [np.pi * i / 4 for i in range(8)]
    #     stripe_counts = [np.mean([self.count_stripes(img, theta=t) for t in thetas]) for img in cluster_images]
    #     return np.mean(stripe_counts)

    def analyze_cluster(self, labels, cluster_label):
        cluster_stripes = [stripe for idx, stripe in enumerate(self.max_stripes_list) if labels[idx] == cluster_label]
        if cluster_stripes:
            mean_stripe_count = np.mean(cluster_stripes)
        else:
            mean_stripe_count = 0
        return mean_stripe_count


    def segment_czm(self):
        ring_sizes_ = [(self.min_range_z2_ - self.min_range_) / self.num_rings_each_zone[0],
                    (self.min_range_z3_ - self.min_range_z2_) / self.num_rings_each_zone[1],
                    (self.min_range_z4_ - self.min_range_z3_) / self.num_rings_each_zone[2],
                    (self.max_range_ - self.min_range_z4_) / self.num_rings_each_zone[3]]

        sector_sizes_ = [2 * math.pi / self.num_sectors_each_zone[0], 
                        2 * math.pi / self.num_sectors_each_zone[1], 
                        2 * math.pi / self.num_sectors_each_zone[2], 
                        2 * math.pi / self.num_sectors_each_zone[3]]

        sector_indices = [] 

        czm = [[
            [Zone() for _ in range(self.num_sectors_each_zone[zone_idx])]
            for _ in range(self.num_rings_each_zone[zone_idx])
        ] for zone_idx in range(self.num_zones)]

        for pt in self.point_cloud:
            x, y, z = pt
            r = xy2radius(x, y)
            if r <= self.max_range_ and r > self.min_range_:
                theta = xy2theta(x, y)
                zone_idx = ring_idx = sector_idx = 0

                if r < self.min_range_z2_:
                    zone_idx = 0
                    ring_idx = min(int((r - self.min_range_) / ring_sizes_[0]), self.num_rings_each_zone[0]-1)
                    sector_idx = min(int(theta / sector_sizes_[0]), self.num_sectors_each_zone[0]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z])
                elif r < self.min_range_z3_:
                    zone_idx = 1
                    ring_idx = min(int((r - self.min_range_z2_) / ring_sizes_[1]), self.num_rings_each_zone[1]-1)
                    sector_idx = min(int(theta / sector_sizes_[1]), self.num_sectors_each_zone[1]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z])
                elif r < self.min_range_z4_:
                    zone_idx = 2
                    ring_idx = min(int((r - self.min_range_z3_) / ring_sizes_[2]), self.num_rings_each_zone[2]-1)
                    sector_idx = min(int(theta / sector_sizes_[2]), self.num_sectors_each_zone[2]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z])
                else:
                    zone_idx = 3
                    ring_idx = min(int((r - self.min_range_z4_) / ring_sizes_[3]), self.num_rings_each_zone[3]-1)
                    sector_idx = min(int(theta / sector_sizes_[3]), self.num_sectors_each_zone[3]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z])
                    
                czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z])
                sector_indices.append((zone_idx, ring_idx, sector_idx)) # 인덱스 추가
        return czm, sector_indices
        #sector_indices 는 각 포인트마다 몇 번째 섹터에 속하는지 나타냄. 

    def process_sectors(self, sectors, img_dim=15, MIN_POINTS=10, MAX_Z_DIFF=0.8): # MAX_Z_DIFF 파라미터 추가
        images = []
        traversable_points = []
        all_image_vectors = []
        image_sector_indices = []

        for zone_idx, layer in enumerate(sectors):
            for ring_idx, ring in enumerate(layer):
                for sector_idx, sector in enumerate(ring):
                    if len(sector.points) > MIN_POINTS:
                        points = np.array([pt[:3] for pt in sector.points])
                        normal_vector = calculate_normal_vector(points)

                        sector_center_x = np.mean(points[:, 0])
                        sector_center_y = np.mean(points[:, 1])
                        sector_center_z = np.mean(points[:, 2])

                        z_values = [pt[2] for pt in sector.points]
                        min_z, max_z = min(z_values), max(z_values)

                        # z 좌표의 최대값과 최소값의 차이가 일정 이하인 경우만 처리
                        if angle_with_z_axis(normal_vector) >= 1.1 and angle_with_z_axis(normal_vector) <= 1.9 and (max_z - min_z) <= MAX_Z_DIFF:
                            traversable_points.extend(points)
                            image = np.zeros((img_dim, img_dim), dtype=np.uint8)

                            x_values = [pt[0] for pt in sector.points]
                            y_values = [pt[1] for pt in sector.points]

                            min_x, max_x = min(x_values), max(x_values)
                            min_y, max_y = min(y_values), max(y_values)

                            for pt in sector.points:
                                normalized_x = int((pt[0] - min_x) / (max_x - min_x) * (img_dim - 1))
                                normalized_y = int((pt[1] - min_y) / (max_y - min_y) * (img_dim - 1))
                                normalized_z = int((pt[2] - min_z) / (max_z - min_z) * 255)

                                image[normalized_y, normalized_x] = normalized_z

                            image_2d = image.reshape(img_dim, img_dim)
                            image_2d = self.remove_specific_pattern(image_2d,15)

                            images.append(image_2d)
                            image_sector_indices.append((zone_idx, ring_idx, sector_idx)) # 해당 이미지의 sector 인덱스 추가

                            image_vector = image_2d.reshape(-1, 1)
                            all_image_vectors.append(image_vector)

        return images, image_sector_indices

    def visualize_clusters(self, images, sector_indices, n_clusters, base_output_folder):
        for img, sector_index in zip(images, sector_indices):
            self.extract_features(img, sector_index)
        print("length of images :", len(images))     
        print("length of mst : " , len(self.max_stripes_list))  
        print(self.max_stripes_list)

        kmeans = KMeans(n_clusters=n_clusters).fit(np.array(self.max_stripes_list).reshape(-1, 1))
        # kmeans = KMeans(n_clusters=n_clusters).fit(X)
        # stripe_counts = [self.analyze_cluster([img])[0] for img in images]

        sector_cluster_mapping = {}

        # Save images to clusters
        for i, label in enumerate(kmeans.labels_):
            max_stripes = self.max_stripes_list[i]
            best_image = self.images_to_cluster[i].reshape(images[i].shape)
            combined_image = np.hstack((images[i], best_image))

            cluster_path = os.path.join(base_output_folder, f"cluster_{label}")
            save_path = os.path.join(cluster_path, f"{label}_stripes_{max_stripes}_{i}.png")  # 파일 이름에 max_stripes 추가
            if not os.path.exists(cluster_path):
                os.makedirs(cluster_path)
            # cv2.imwrite(os.path.join(cluster_path, f"image_{i}.png"), images[i])
            # cv2.imwrite(save_path, combined_image)

            # Map sector indices to clusters
            sector_index = sector_indices[i]
            sector_cluster_mapping[sector_index] = label

        # Analyze clusters
        cluster_stripe_counts = {}
        for i in range(n_clusters):
            cluster_images = [img for idx, img in enumerate(images) if kmeans.labels_[idx] == i]
            cluster_stripe_counts = {i: self.analyze_cluster(kmeans.labels_, i) for i in range(n_clusters)}

        # Sort clusters by mean stripe count
        sorted_clusters = sorted(cluster_stripe_counts.items(), key=lambda x: x[1], reverse=True)
        print(sorted_clusters)

        # Visualize clusters
        for cluster_index, _ in sorted_clusters:
            cluster_path = os.path.join(base_output_folder, f"cluster_{cluster_index}")
            image_files = [os.path.join(cluster_path, f) for f in os.listdir(cluster_path) if os.path.isfile(os.path.join(cluster_path, f))]
            images = [cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files]

            # plt.figure(figsize=(10, 10))
            # for idx, img in enumerate(images[:16]): # Displaying the first 16 images from each cluster
            #     plt.subplot(4, 4, idx + 1)
            #     plt.imshow(img, cmap='gray')
            #     plt.axis('off')

            # plt.suptitle(f"Cluster {cluster_index}")
            # plt.show()
        # print("Clusters have been visualized by stripe pattern repetition.")

        # for i in range(n_clusters):
        #     cluster_images = [img for idx, img in enumerate(images) if kmeans.labels_[idx] == i]
        #     # cluster_stripe_count_mean = self.analyze_cluster(cluster_images)
        #     print(f"Cluster {i} Mean Stripe Count: {cluster_stripe_count_mean}")

        #     # # Show only first 5 Gabor-filtered images per cluster
        #     # for idx, gabor_image in enumerate(gabor_images[:5]):
        #     #     plt.imshow(gabor_image, cmap='gray')
        #     #     plt.title(f'Cluster {i} Gabor Image {idx}')
        #     #     plt.show()
        return sorted_clusters, sector_cluster_mapping

    def calculate_cluster_z_variance(self, sectors, sector_cluster_mapping):
        cluster_z_variances = {}
        cluster_z_variance_means = {}  # 클러스터별 z 분산의 평균을 저장할 딕셔너리

        # 클러스터별 z 값 분산을 계산
        for zone_idx, zone in enumerate(sectors):
            for ring_idx, ring in enumerate(zone):
                for sector_idx, sector in enumerate(ring):
                    cluster_label = sector_cluster_mapping.get((zone_idx, ring_idx, sector_idx))
                    if cluster_label is not None:
                        z_values = [pt[2] for pt in sector.points]
                        z_variance = np.var(z_values)

                        if cluster_label not in cluster_z_variances:
                            cluster_z_variances[cluster_label] = []

                        cluster_z_variances[cluster_label].append(z_variance)

        # 클러스터별로 그래프 그리기
        for cluster_label, variances in cluster_z_variances.items():
            # Removing outliers
            Q1 = np.percentile(variances, 10)
            Q3 = np.percentile(variances, 90)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            filtered_variances = [v for v in variances if v >= lower_bound and v <= upper_bound]

            # plt.figure(figsize=(10, 4))
            # plt.bar(range(len(filtered_variances)), filtered_variances)
            # plt.xlabel('Sector')
            # plt.ylabel('Z Variance')
            # plt.title(f'Cluster {cluster_label} - Z Variance by Sector (Outliers Removed)')
            # plt.show()

            # 클러스터별 z 분산의 평균 계산
            cluster_z_variance_means[cluster_label] = np.mean(filtered_variances)

        cluster_z_variance_means = sorted(cluster_z_variance_means.items(), key=lambda x: x[1])
        print(cluster_z_variance_means)
        return cluster_z_variance_means


In [249]:
pcd_file_path = "/media/rtlink/JetsonSSD-256/Download/Rellis_3D_os1_cloud_node_color_ply/Rellis-3D/00000/os1_cloud_node_color_ply/pcd_files/frame001896-1581624842_364.pcd"
czm = CZM(pcd_file_path) # Zone 대신 CZM 클래스를 사용

-1.6572240591049194 5.804795742034912


In [250]:
sectors, sector_indices = czm.segment_czm() # 인스턴스 czm을 통해 메서드를 호출하고, sector_indices도 받습니다.

In [251]:
images, image_sector_indices = czm.process_sectors(sectors) # 인스턴스 czm을 통해 메서드를 호출하고, image_sector_indices도 받습니다.

In [252]:
sorted_clusters, sector_cluster_mapping = czm.visualize_clusters(images, image_sector_indices, n_clusters=3, base_output_folder='output_clusters') # 인스턴스 czm을 통해 메서드를 호출

length of images : 29
length of mst :  29
[6, 5, 7, 5, 8, 8, 6, 8, 6, 5, 6, 7, 7, 4, 5, 2, 5, 8, 6, 7, 3, 6, 7, 1, 5, 5, 4, 5, 1]
[(1, 7.444444444444445), (0, 5.25), (2, 1.75)]


/home/rtlink/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [253]:
mean_cluster_z_variances = czm.calculate_cluster_z_variance(sectors, sector_cluster_mapping)

[(1, 0.0011475111735187022), (0, 0.0020327932502873865), (2, 0.023261160205678658)]


In [259]:
def calculate_order_percentage(pcd_directory_path):
    matching_count = 0
    total_count = 0

    x1_values, y1_values, z1_values = [], [], []
    x2_values, y2_values, z2_values = [], [], []

    pcd_files = list(Path(pcd_directory_path).glob('*.pcd'))[:50]
    for pcd_file_path in pcd_files:
        czm = CZM(str(pcd_file_path))
        sectors, sector_indices = czm.segment_czm()
        images, image_sector_indices = czm.process_sectors(sectors)
        sorted_clusters, sector_cluster_mapping = czm.visualize_clusters(images, image_sector_indices, n_clusters=3, base_output_folder='output_clusters') # base_output_folder 추가
        mean_cluster_z_variances = czm.calculate_cluster_z_variance(sectors, sector_cluster_mapping)

        # sorted_clusters와 mean_cluster_z_variances의 첫 번째 항목 추출
        sorted_clusters_order = [item[0] for item in sorted_clusters]
        sorted_clusters_order_2 = [item[1] for item in sorted_clusters]
        print(sorted_clusters_order)
        mean_cluster_order = [item[0] for item in mean_cluster_z_variances]
        mean_cluster_order_2 = [item[1] for item in mean_cluster_z_variances]
        print(mean_cluster_order)

        x1_values.append(sorted_clusters_order_2[0])
        x2_values.append(mean_cluster_order_2[0]*100)
        y1_values.append(sorted_clusters_order_2[1])
        y2_values.append(mean_cluster_order_2[1]*100)
        z1_values.append(sorted_clusters_order_2[2])
        z2_values.append(mean_cluster_order_2[2]*100)


        if sorted_clusters_order == mean_cluster_order:
            matching_count += 1

        total_count += 1

    percentage = (matching_count / total_count) * 100

    correlation_x = np.corrcoef(x1_values, x2_values)[0, 1]
    print(f"Correlation between x1 and x2: {correlation_x}")

    correlation_y = np.corrcoef(y1_values, y2_values)[0, 1]
    print(f"Correlation between y1 and y2: {correlation_y}")

    correlation_z = np.corrcoef(z1_values, z2_values)[0, 1]
    print(f"Correlation between z1 and z2: {correlation_z}")

    return percentage

pcd_directory_path = "/media/rtlink/JetsonSSD-256/Download/Rellis_3D_os1_cloud_node_color_ply/Rellis-3D/00000/os1_cloud_node_color_ply/pcd_files/"
percentage = calculate_order_percentage(pcd_directory_path)
print(f"Percentage of PCD files with matching orders: {percentage}%")


-2.383615493774414 6.761868000030518
length of images : 17
length of mst :  17
[3, 6, 9, 3, 7, 6, 6, 4, 5, 7, 7, 6, 9, 3, 5, 6, 2]
[(2, 9.0), (0, 6.1), (1, 3.0)]
[(2, 0.0005381917081214545), (0, 0.0018443671703626883), (1, 0.003989699841962877)]
[2, 0, 1]
[2, 0, 1]
-2.495469331741333 7.282830715179443


/home/rtlink/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


length of images : 44
length of mst :  44
[7, 7, 6, 6, 6, 6, 7, 7, 5, 6, 5, 5, 6, 5, 5, 4, 5, 7, 6, 6, 7, 4, 7, 7, 6, 6, 5, 6, 6, 6, 5, 6, 6, 7, 3, 8, 4, 5, 5, 7, 4, 7, 6, 3]
[(2, 7.083333333333333), (0, 5.615384615384615), (1, 3.6666666666666665)]
[(2, 0.0011019100334224467), (0, 0.002277702733388997), (1, 0.019936472834647122)]
[2, 0, 1]
[2, 0, 1]
-3.5213091373443604 7.311774730682373


/home/rtlink/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


length of images : 190
length of mst :  190
[6, 6, 7, 5, 4, 5, 8, 7, 7, 8, 5, 5, 4, 4, 7, 2, 5, 4, 4, 6, 4, 7, 7, 5, 5, 7, 6, 6, 4, 5, 7, 5, 5, 6, 7, 5, 4, 5, 6, 5, 4, 5, 3, 6, 2, 7, 4, 4, 6, 5, 6, 6, 5, 5, 5, 5, 8, 5, 4, 5, 6, 5, 6, 8, 8, 6, 6, 6, 7, 8, 7, 5, 7, 6, 6, 6, 4, 7, 4, 6, 5, 5, 5, 6, 6, 7, 5, 6, 6, 7, 7, 5, 5, 5, 6, 5, 6, 7, 8, 6, 5, 5, 5, 6, 6, 4, 3, 7, 6, 6, 6, 6, 6, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 5, 6, 5, 8, 6, 5, 6, 5, 6, 6, 9, 5, 6, 6, 6, 6, 9, 5, 4, 3, 5, 4, 5, 7, 7, 3, 3, 6, 1, 6, 6, 5, 5, 8, 5, 6, 3, 4, 5, 4, 6, 6, 4, 6, 6, 6, 5, 3, 5, 3, 4, 4, 3, 5, 4, 5, 7, 3, 4, 7, 5, 3, 1, 1, 3, 3, 3]
[(0, 7.382352941176471), (1, 5.4774774774774775), (2, 3.3777777777777778)]
[(0, 0.0006310881316838135), (1, 0.001812468207364933), (2, 0.003437316738605002)]
[0, 1, 2]
[0, 1, 2]
-3.0920188426971436 7.2586164474487305


...

/home/rtlink/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


length of images : 51
length of mst :  51
[5, 6, 9, 6, 6, 1, 5, 6, 6, 6, 3, 5, 4, 5, 4, 5, 7, 6, 9, 5, 9, 6, 9, 6, 6, 9, 3, 3, 5, 7, 7, 8, 7, 5, 4, 6, 6, 3, 4, 4, 8, 3, 4, 3, 5, 2, 5, 1, 1, 5, 1]
[(1, 8.090909090909092), (0, 5.206896551724138), (2, 2.1818181818181817)]
[(1, 0.0027139530975026772), (0, 0.0039434038963871475), (2, 0.011567696726771893)]
[1, 0, 2]
[1, 0, 2]
-3.3223280906677246 7.653062343597412


/home/rtlink/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


length of images : 44
length of mst :  44
[8, 3, 2, 3, 7, 9, 6, 6, 4, 4, 3, 5, 4, 3, 2, 10, 5, 7, 7, 6, 6, 6, 6, 6, 4, 5, 5, 5, 5, 4, 7, 2, 1, 8, 0, 3, 3, 4, 3, 3, 6, 3, 5, 1]
[(0, 7.875), (2, 5.095238095238095), (1, 2.3333333333333335)]
[(0, 0.0029351832917940165), (2, 0.0065747632261384924), (1, 0.018652487868379863)]
[0, 2, 1]
[0, 2, 1]
Correlation between x1 and x2: -0.10490855690166483
Correlation between y1 and y2: -0.46105833250406236
Correlation between z1 and z2: -0.3469659244114352
Percentage of PCD files with matching orders: 84.0%


/home/rtlink/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
